In [1]:
#-------------------
# Printer decorator
#-------------------
def _decorator(func):
    def wrap(*args, **kwargs):
        print('#'+'-'*(len(*args)+2))
        func(*args)
        print('#'+'-'*(len(*args)+2))
    return wrap
@_decorator
def printing(msg):
    print('# '+msg)

In [2]:
#@title CowSay
def _decorator(func):
    def wrap(*args, **kwargs):
        print(' '+'-'*(len(*args)+2))
        func(*args)
        print(' '+'-'*(len(*args)+2))
        print(r"""
        \   ^__^
         \  (oo)\_______
            (__)\       )\/
                ||----w |
                ||     ||
        """)
    return wrap
@_decorator
def cowsay(msg):
    print('<\033[96m '+msg+'\033[00m >')

In [3]:
#-- Import modules --#
import plotly.express as px
import pandas as pd
import numpy as np

In [4]:
#-- Read Gen Dataset --#
dataset = pd.read_parquet('../dataset/currentGen.parquet')
dataset

,date,NombreEmpresa,Tipoinfoabrev,power,name
0,2025-09-08 00:30:00,ENGIE,MW,197.5275,W.F. Punta Lomitas
1,2025-09-08 01:00:00,ENGIE,MW,197.5275,W.F. Punta Lomitas
2,2025-09-08 01:30:00,ENGIE,MW,196.5675,W.F. Punta Lomitas
3,2025-09-08 02:00:00,ENGIE,MW,196.5675,W.F. Punta Lomitas
4,2025-09-08 02:30:00,ENGIE,MW,197.7335,W.F. Punta Lomitas
...,...,...,...,...,...
955,2025-09-09 22:00:00,ORYGEN PERU S.A.A.,MW,83.1575,W.F. Wayra I
956,2025-09-09 22:30:00,ORYGEN PERU S.A.A.,MW,73.6690,W.F. Wayra I
957,2025-09-09 23:00:00,ORYGEN PERU S.A.A.,MW,73.6690,W.F. Wayra I
958,2025-09-09 23:30:00,ORYGEN PERU S.A.A.,MW,65.0355,W.F. Wayra I


In [5]:
#-- Import modules --#
import dash
import pandas as pd
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
#=======================
#    Read Gen Dataset 
#=======================
dataset = pd.read_parquet('../dataset/currentGen.parquet')
#========================
# App & Theme Setup
#========================
app = Dash(
    __name__,
    external_stylesheets=[
        # keep icons only; DO NOT preload a dbc theme here
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
    ],
    assets_folder='/content/assets',
)
app.title = "ENGIE WindForecast Platform "

# Choose distinct themes
LIGHT_THEME = dbc.themes.CERULEAN
DARK_THEME  = dbc.themes.SUPERHERO
#========================
# UI Bits
#========================
def create_metric_card(icon, title, value, change, color="primary"):
    up = (str(change).startswith("+") or float(str(change).replace("+","").replace("%","")) >= 0)
    trend_class = "text-success" if up else "text-danger"
    change_txt  = f"{change}% from last month" if "%" not in str(change) else f"{change} from last month"
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([html.I(className=f"fas {icon} fa-2x text-{color}")],
                         className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0", style={'font-size': '32px','font-weight': 'bold'}),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(change_txt, className=trend_class)
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-sun", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-moon", html_for="switch"),
    ],
    className="my-2"
)

#========================
# Layout
#========================
app.layout = dbc.Container([
    # Theme link and state (this is what we toggle)
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),

    dbc.Row([
        # Left Panel
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={'width': '200px','margin-bottom': '20px','display': 'block','margin-left': 'auto','margin-right': 'auto'}
                ),
                html.H1([html.Span("Welcome"), html.Br(), html.Span("WindForecast Platform")],
                        style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation "
                    "insights for 10 wind farms and the SEIN system. It supports short and "
                    "long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value":1},
                        {"label": "Long  Term (07 months)", "value":2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[
                        {"label": "SEIN", "value": "0"},
                        {"label": "W.F. Punta Lomitas", "value": "1"},
                        {"label": "W.F. Cupisnique", "value": "2"},
                        {"label": "W.F. Duna", "value": "3"},
                        {"label": "W.F. Huambos", "value": "4"},
                        {"label": "W.F. Marcona", "value": "5"},
                        {"label": "W.F. San Juan", "value": "6"},
                        {"label": "W.F. Talara", "value": "7"},
                        {"label": "W.F. Tres Hermanas", "value": "8"},
                        {"label": "W.F. Wayra Ext", "value": "9"},
                        {"label": "W.F. Wayra I", "value": "10"},
                    ],
                    value="1",
                    className="mb-3"
                ),
            ])
        ], width=4),

        # Right Panel
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            dcc.Graph(id='genPlot', style={'height':'250px'}),
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
], fluid=True)

#========================
# Callbacks
#========================
@app.callback(
    Output("theme-link", "href"),
    Output("theme-store", "data"),
    Input("switch", "value"),
    State("theme-store", "data"),
)
def toggle_theme(switch_value, data):
    dark = bool(switch_value)
    theme_href = DARK_THEME if dark else LIGHT_THEME
    return theme_href, {"dark": dark}

@app.callback(
    Output('genPlot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input("switch", "value"),
)
def updateGenPlot(windfarm_value, dark):
    # Example data hook (replace 'dataset' with your actual DataFrame in scope)
    windfarm_mapping = {
        "0": "SEIN", "1": "W.F. Punta Lomitas", "2": "W.F. Cupisnique",
        "3": "W.F. Duna", "4": "W.F. Huambos", "5": "W.F. Marcona",
        "6": "W.F. San Juan", "7": "W.F. Talara", "8": "W.F. Tres Hermanas",
        "9": "W.F. Wayra Ext", "10": "W.F. Wayra I"
    }
    windfarm_name = windfarm_mapping.get(windfarm_value, "SEIN")
    subset = dataset.query("name==@windfarm_name").copy()  # <- ensure 'dataset' exists

    colors = {
        'background': '#0f2537' if dark else '#ffffff',
        'text': '#ffffff' if dark else '#2c3e50',
    }

    fig = px.line(subset, x='date', y='power', line_shape='linear')
    fig.update_traces(line=dict(width=1.2))
    fig.update_layout(
        template="plotly_dark" if dark else "plotly_white",
        xaxis=dict(
            tickfont_size=9,
            showgrid=True, gridcolor='lightgrey', gridwidth=0.01, tickangle=0,
            tickformat="%d-%b\n%H:%M hrs.", dtick=18*3600000, griddash='solid'
        ),
        yaxis=dict(showgrid=True, gridcolor='lightgrey', tickfont_size=10,
                   gridwidth=0.01, griddash='solid', 
                   title=dict(text='Wind Generation (MW)', font=dict(size=12))
                   ),
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font=dict(color=colors['text']),
        xaxis_title='',
        # yaxis_title='Wind Generation (MW)',
        # width=1000, height=300,
        legend=dict(
            orientation='h', yanchor='bottom', y=1.05, xanchor='right', x=.8,
            tracegroupgap=5, itemclick='toggle', itemdoubleclick='toggle'
        ),
        hovermode="x unified",
        margin=dict(t=10, b=5),
        # margin=dict(l=10, r=10, t=10, b=30),
        autosize=True,
    )
    # --- range selector styles based on theme ---
    rs_style = dict(
        bgcolor="#000000" if dark else "#ffffff",
        activecolor="#4169e1" if dark else "#00bdff",  # royalblue / dodgerblue
        font=dict(color="#ffffff" if dark else "#000000"),
    )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1D", step="day", stepmode="backward"),
                dict(count=5, label="5D", step="day", stepmode="backward"),
                dict(step="all")
            ],
            **rs_style
        )
    )
    return fig
if __name__ == '__main__':
    # If you're in a notebook, use JupyterDash. Otherwise:
    app.run(debug=False, port=8052)

In [55]:
dataset.query("name=='W.F. Punta Lomitas'")

,date,NombreEmpresa,Tipoinfoabrev,power,name
0,2025-09-08 00:30:00,ENGIE,MW,197.5275,W.F. Punta Lomitas
1,2025-09-08 01:00:00,ENGIE,MW,197.5275,W.F. Punta Lomitas
2,2025-09-08 01:30:00,ENGIE,MW,196.5675,W.F. Punta Lomitas
3,2025-09-08 02:00:00,ENGIE,MW,196.5675,W.F. Punta Lomitas
4,2025-09-08 02:30:00,ENGIE,MW,197.7335,W.F. Punta Lomitas
...,...,...,...,...,...
91,2025-09-09 22:00:00,ENGIE,MW,138.3560,W.F. Punta Lomitas
92,2025-09-09 22:30:00,ENGIE,MW,131.3810,W.F. Punta Lomitas
93,2025-09-09 23:00:00,ENGIE,MW,131.3810,W.F. Punta Lomitas
94,2025-09-09 23:30:00,ENGIE,MW,123.3385,W.F. Punta Lomitas


In [5]:
dataset['name'].unique()

array(['W.F. Punta Lomitas', 'W.F. Cupisnique', 'W.F. Duna',
       'W.F. Huambos', 'W.F. Marcona', 'W.F. San Juan', 'W.F. Talara',
       'W.F. Tres Hermanas', 'W.F. Wayra Ext', 'W.F. Wayra I'],
      dtype=object)

In [8]:
#-- Import modules --#
import dash
import time
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px

#-- Layout --#
app = Dash(__name__,
           external_stylesheets=[
               dbc.themes.SUPERHERO,
               "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"],
           assets_folder='/content/assets',
           )
#-- Tab Name --#
app.title = "ENGIE WindForecast Platform "
#-- Theme to Toggle --#
# LIGHT_THEME = dbc.themes.CERULEAN
LIGHT_THEME = dbc.themes.SUPERHERO
# LIGHT_THEME = dbc.themes.CERULEAN
DARK_THEME = dbc.themes.SUPERHERO
#-- Cards --#
def create_metric_card(icon, title, value, change, color="primary"):
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([
                    html.I(className=f"fas {icon} fa-2x text-{color}")
                ], className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0",
                            style={
                                'font-size': '32px',
                                # 'font-family': 'DidoneRoomNumbers',
                                'font-weight': 'bold',
                                }),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(f"{change}% from last month",
                              className=f"text-{'success' if '+' in change else 'danger'}")
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")
metric_cards = [
                create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
                create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
                create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
                ]
#-- Function to create sample figures --#
"""
  All plots on the right panel
"""
#-- UI Theme switch --#
color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-moon", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-sun", html_for="switch"),
    ],
    className="my-2"
)
#-- Main Layout --#
app.layout = dbc.Container([
    #-- Wait for the intro --#
    #-- Theme link and store --#
    # dbc.Label(className="fa fa-moon", html_for="switch"),
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),
    #-- Left Panel --#
    dbc.Row([
        dbc.Col([
            html.Div([
                #-- The switch --#
                html.Div(color_mode_switch, className='mb-3 text-end'),
                #-- Logo --#
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={
                        'width': '200px',
                        'margin-bottom': '20px',
                        'display': 'block',
                        'margin-left': 'auto',
                        'margin-right': 'auto'
                    }
                ),
                #-- Title --#
                html.H1([
                    html.Span("Welcome"),
                    html.Br(),
                    html.Span("WindForecast Platform")
                ],
                        style={'color':'#00bdff'},
                        className="text-center mb-4"),
                #-- Description --#
                html.P(
                    "\
                      This platform provides accurate wind forecasts and energy generation \
                      insights for 10 wind farms and the SEIN system. It supports short and \
                      long-term planning with interactive tools and clear visualizations. \
                    ",
                    className="text-justify mb-4",
                ),
                #-- Time range buttons --#
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value":1,},
                        {"label": "Long  Term (07 months)", "value":2},
                    ],
                    value=1,
                ),
                html.Hr(),
                #-- Wind Farms Section --#
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'},),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[
                        {"label": "SEIN", "value": "SEIN"},
                        {"label": "W.F. Punta Lomitas", "value": "1"},
                        {"label": "W.F. Cupisnique", "value": "2"},
                        {"label": "W.F. Duna", "value": "3"},
                        {"label": "W.F. Huambos", "value": "4"},
                        {"label": "W.F. Marcona", "value": "5"},
                        {"label": "W.F. San Juan", "value": "6"},
                        {"label": "W.F. Talara", "value": "7"},
                        {"label": "W.F. Tres Hermanas", "value": "8"},
                        {"label": "W.F. Wayra Ext.", "value": "9"},
                        {"label": "W.F. Wayra I", "value": "10"},
                    ],
                    value="0",
                    className="mb-3"
                ),
            ])
        #-- Set the width left panel --#
        ], width=4),
        #-- Right Panel --#
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'},),
            dbc.Row([
                dbc.Col(card, width=4) for card in metric_cards
                ]),
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'},),
            #-- Add the generation plot --#
            dcc.Graph(id='genPlot', style={'height':'400px'}),
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'},),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'},),
        ], width=8)
    ])
],
  fluid=True,
  # className='dashboard-container',

  # style={
  #     'width':'1400px',
  #     'height':'2200px',
  #     'display': 'flex',

  #     }
)
@app.callback(
    Output('genPlot', 'figure'),
    Input('windfarm-buttons', 'label'),
)
def updateGenPlot(windfarm):
    print(windfarm)
    # subset = dataset.query("name==@windfarm").copy()
    subset = dataset.query("name=='W.F. Punta Lomitas'")
    fig = px.line(subset, x='date', y='power')
    return fig

if __name__ == '__main__':
    app.run(mode='inline', debug=False)

None


In [41]:
#-- Simple Plot --#
sname = dataset['name'].unique()[3]
fig = px.line(dataset.query("name==@sname"), x='date', y='power', line_shape='spline')
fig.update_layout(xaxis_title='', yaxis_title='Wind Generation (MW)')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward", ),
            dict(count=5, label="5d", step="day", stepmode="backward"),
            dict(count=10, label="10d", step="day", stepmode="todate"),
            dict(step="all")
        ])
    )
)
#-- Main Layout --#
fig.update_layout(
    title_text=f"<br><sup>Monthly average – Simulated Data </sup></br>",
    title_font=dict(size=24, family="Arial", color="black"),
    template="plotly_white",
    xaxis_title='',
    width=1000,
    height=500,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.05,
        xanchor='right',
        x=.8,
        tracegroupgap=5,
        itemclick='toggle',
        itemdoubleclick='toggle'
    ),
    hovermode="x unified",
    margin=dict(t=180, b=80)
)
# fig.update_traces(line=dict(dash="dot", width=1.2))
fig

### **2. Check my App**

In [7]:
# -*- coding:utf-8 -*-
#-----------------------------------------------------
# @Project: ./windShortTermForecast/app/
# @File: ./windShortTermForecast/app/app.py
# @Author: Carlos Enciso Ojeda
# @Email: carlos.enciso.o@gmail.com
#-----------------------------------------------------
#-- Import modules --#
import dash
import time
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

#-- Function to generate sample data for demonstration --#
def generate_sample_data(windfarm_id):
    """Generate sample wind generation data for different wind farms"""
    np.random.seed(int(windfarm_id))  # For consistent results
    
    # Create date range for the next 15 days
    dates = [datetime.now() + timedelta(hours=i) for i in range(24*15)]
    
    # Different generation patterns based on windfarm
    if windfarm_id == "0":  # SEIN
        base_power = np.random.uniform(800, 1200, len(dates))
        daily_pattern = np.sin(np.linspace(0, 4*np.pi, len(dates))) * 200
        noise = np.random.normal(0, 50, len(dates))
        generation = base_power + daily_pattern + noise
        farm_name = "SEIN System"
    else:
        # Individual wind farms have different patterns
        base_multiplier = int(windfarm_id) * 50
        base_power = np.random.uniform(100 + base_multiplier, 200 + base_multiplier, len(dates))
        daily_pattern = np.sin(np.linspace(0, 4*np.pi, len(dates))) * (50 + int(windfarm_id)*5)
        noise = np.random.normal(0, 10 + int(windfarm_id), len(dates))
        generation = base_power + daily_pattern + noise
        farm_name = f"W.F. {['Punta Lomitas', 'Cupisnique', 'Duna', 'Huambos', 'Marcona', 'San Juan', 'Talara', 'Tres Hermanas', 'Wayra Ext.', 'Wayra I'][int(windfarm_id)-1]}"
    
    return pd.DataFrame({
        'datetime': dates,
        'generation_mw': generation,
        'windfarm': farm_name
    })

#-- Layout --#
app = Dash(__name__,
           external_stylesheets=[
               dbc.themes.SUPERHERO,
               "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"],
           assets_folder='/content/assets',
           )
#-- Tab Name --#
app.title = "ENGIE WindForecast Platform "

#-- Theme to Toggle --#
LIGHT_THEME = dbc.themes.SUPERHERO
DARK_THEME = dbc.themes.SUPERHERO

#-- Cards --#
def create_metric_card(icon, title, value, change, color="primary"):
    return dbc.Card([
        dbc.CardBody([
            html.Div([
                html.Div([
                    html.I(className=f"fas {icon} fa-2x text-{color}")
                ], className="col-3 d-flex align-items-center"),
                html.Div([
                    html.H4(value, className="mb-0",
                            style={
                                'font-size': '32px',
                                'font-weight': 'bold',
                                }),
                    html.P(title, className="mb-0 text-muted"),
                    html.Small(f"{change}% from last month",
                              className=f"text-{'success' if '+' in change else 'danger'}")
                ], className="col-9")
            ], className="row align-items-center")
        ])
    ], className="mb-3 shadow-sm")

metric_cards = [
    create_metric_card("fa-solid fa-wind", "m/s", "13.5", "+2.5", "primary"),
    create_metric_card("fa-solid fa-bolt", "MW", "1,234", "+900.5", "info"),
    create_metric_card("fa-solid fa-chart-line", "MAPE (%)", "20.2", "20", "warning"),
]

#-- UI Theme switch --#
color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-moon", html_for="switch"),
        dbc.Switch(id="switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-sun", html_for="switch"),
    ],
    className="my-2"
)

#-- Main Layout --#
app.layout = dbc.Container([
    #-- Theme link and store --#
    html.Link(rel="stylesheet", href=LIGHT_THEME, id="theme-link"),
    dcc.Store(id="theme-store", data={"dark": False}),
    
    #-- Left Panel --#
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Div(color_mode_switch, className='mb-3 text-end'),
                html.Img(
                    src='https://upload.wikimedia.org/wikipedia/commons/8/8f/Logo-engie.svg',
                    style={
                        'width': '200px',
                        'margin-bottom': '20px',
                        'display': 'block',
                        'margin-left': 'auto',
                        'margin-right': 'auto'
                    }
                ),
                html.H1([
                    html.Span("Welcome"),
                    html.Br(),
                    html.Span("WindForecast Platform")
                ], style={'color':'#00bdff'}, className="text-center mb-4"),
                html.P(
                    "This platform provides accurate wind forecasts and energy generation \
                    insights for 10 wind farms and the SEIN system. It supports short and \
                    long-term planning with interactive tools and clear visualizations.",
                    className="text-justify mb-4",
                ),
                html.H5("Forecast Range", className="mb-3"),
                dbc.RadioItems(
                    id='top-buttons',
                    className='btn-group-horizontal',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-primary",
                    labelCheckedClassName="btn btn-primary",
                    options=[
                        {"label": "Short Term (15 days)", "value":1},
                        {"label": "Long  Term (07 months)", "value":2},
                    ],
                    value=1,
                ),
                html.Hr(),
                html.H5("WindFarms", className="mt-4", style={'color':'#00bdff'}),
                dbc.RadioItems(
                    id="windfarm-buttons",
                    options=[
                        {"label": "SEIN", "value": "0"},
                        {"label": "W.F. Punta Lomitas", "value": "1"},
                        {"label": "W.F. Cupisnique", "value": "2"},
                        {"label": "W.F. Duna", "value": "3"},
                        {"label": "W.F. Huambos", "value": "4"},
                        {"label": "W.F. Marcona", "value": "5"},
                        {"label": "W.F. San Juan", "value": "6"},
                        {"label": "W.F. Talara", "value": "7"},
                        {"label": "W.F. Tres Hermanas", "value": "8"},
                        {"label": "W.F. Wayra Ext.", "value": "9"},
                        {"label": "W.F. Wayra I", "value": "10"},
                    ],
                    value="0",
                    className="mb-3"
                ),
            ])
        ], width=4),
        
        #-- Right Panel --#
        dbc.Col([
            html.Br(),
            html.H4("Monitoring KPI", className="mt-4", style={'color':'#00bdff'}),
            dbc.Row([dbc.Col(card, width=4) for card in metric_cards]),
            
            html.H4("Forecast Wind Generation", className="mt-4", style={'color':'#00bdff'}),
            #-- Add the generation plot here --#
            dcc.Graph(id='generation-plot', style={'height': '400px'}),
            
            html.H4("Forecast Wind Resource", className="mt-4", style={'color':'#00bdff'}),
            html.H4("Resume Forecast Skill", className="mt-4", style={'color':'#00bdff'}),
        ], width=8)
    ])
], fluid=True)

#-- Callback to update the generation plot based on windfarm selection --#
@app.callback(
    Output('generation-plot', 'figure'),
    Input('windfarm-buttons', 'value'),
    Input('top-buttons', 'value')
)
def update_generation_plot(windfarm_id, forecast_range):
    # Generate sample data based on selected windfarm
    df = generate_sample_data(windfarm_id)
    
    # Create the plot using Plotly Express
    fig = px.line(
        df, 
        x='datetime', 
        y='generation_mw',
        title=f'Wind Generation Forecast - {df["windfarm"].iloc[0]}',
        labels={'datetime': 'Date', 'generation_mw': 'Generation (MW)'}
    )
    
    # Customize the plot appearance
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font_color='white',
        xaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray'),
        yaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray'),
        hovermode='x unified'
    )
    
    # Add a range slider for better navigation
    fig.update_xaxes(rangeslider_visible=True)
    
    return fig

if __name__ == '__main__':
    app.run(mode='inline', debug=False)

In [9]:
import plotly.express as px
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
display(df)

fig = px.line(df, x='Date', y='AAPL.High', title='Time Series with Range Slider and Selectors')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_layout(
    title_text=f"<br><sup>Monthly average – Simulated Data </sup></br>",
    title_font=dict(size=24, family="Arial", color="black"),
    template="plotly_white",
    xaxis_title='',
    width=1000,
    height=500,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.05,
        xanchor='right',
        x=.8,
        tracegroupgap=5,
        itemclick='toggle',
        itemdoubleclick='toggle'
    ),
    hovermode="x unified",
    margin=dict(t=180, b=80)
)
fig.show()

,Date,AAPL.Open,AAPL.High,AAPL.Low,AAPL.Close,AAPL.Volume,AAPL.Adjusted,dn,mavg,up,direction
0,2015-02-17,127.489998,128.880005,126.919998,127.830002,63152400,122.905254,106.741052,117.927667,129.114281,Increasing
1,2015-02-18,127.629997,128.779999,127.449997,128.720001,44891700,123.760965,107.842423,118.940333,130.038244,Increasing
2,2015-02-19,128.479996,129.029999,128.330002,128.449997,37362400,123.501363,108.894245,119.889167,130.884089,Decreasing
3,2015-02-20,128.619995,129.500000,128.050003,129.500000,48948400,124.510914,109.785449,120.763500,131.741551,Increasing
4,2015-02-23,130.020004,133.000000,129.660004,133.000000,70974100,127.876074,110.372516,121.720167,133.067817,Increasing
...,...,...,...,...,...,...,...,...,...,...,...
501,2017-02-10,132.460007,132.940002,132.050003,132.119995,20065500,132.119995,114.494004,124.498666,134.503328,Decreasing
502,2017-02-13,133.080002,133.820007,132.750000,133.289993,23035400,133.289993,114.820798,125.205166,135.589534,Increasing
503,2017-02-14,133.470001,135.089996,133.250000,135.020004,32815500,135.020004,115.175718,125.953499,136.731280,Increasing
504,2017-02-15,135.520004,136.270004,134.619995,135.509995,35501600,135.509995,115.545035,126.723499,137.901963,Decreasing
